In [16]:
import pandas as pd

# Remover colunas desnecessarias
# Identificar os usuarios (id) que cometeram fraude (dataset apenas com movimentações de usuarios fraudulentos) 

In [17]:
identity_dataset = pd.read_csv(r'../dataset/train_identity.csv')
transaction_dataset = pd.read_csv(r'../dataset/train_transaction.csv')

In [18]:
# UserID = combinação de card1, addr1, and D1
transaction_dataset['UserID'] = transaction_dataset['card1'].astype(str) + transaction_dataset['addr1'].astype(str) + transaction_dataset['D1'].astype(str)
transaction_dataset['UserID'] = transaction_dataset['UserID'].str.replace('nan', '0').str.replace('.', '')

# Manter apenas as colunas necessárias
transaction_dataset = transaction_dataset[['UserID','TransactionID', 'isFraud', 'TransactionDT', 'TransactionAmt', 'ProductCD', 'card4', 'card6', 'addr1', 'addr2', 'P_emaildomain']]

In [19]:
identity_dataset = identity_dataset[['TransactionID', 'id_30', 'id_31', 'DeviceType', 'DeviceInfo']]
identity_dataset.rename(columns={'id_30': 'SisOp', 'id_31': 'Browser'}, inplace=True)

In [20]:
# Normalizar a coluna SisOp
def normalize_sisop_column(serie):
    os_mapping = {
        'ios': 'iOS',
        'android': 'Android',
        'mac': 'Mac',
        'windows': 'Windows',
        'linux': 'Linux'
    }

    def normalize_os(value):
        value = str(value).lower()
        for key in os_mapping:
            if key in value:
                return os_mapping[key]
        return 'Other'
    
    return serie.apply(normalize_os)

identity_dataset['SisOp'] = normalize_sisop_column(identity_dataset['SisOp'])

In [21]:
# Normalizar a coluna Browser
def normalize_browser_column(serie):
    browser_mapping = {
        'chrome': 'Chrome',
        'firefox': 'Firefox',
        'safari': 'Safari',
        'ie': 'Internet Explorer',
        'edge': 'Edge',
        'samsung': 'Samsung',
        'opera': 'Opera'
    }

    def normalize_browser(value):
        value = str(value).lower()
        for key in browser_mapping:
            if key in value:
                return browser_mapping[key]
        return 'Other'
    
    return serie.apply(normalize_browser)

identity_dataset['Browser'] = normalize_browser_column(identity_dataset['Browser'])


In [22]:
# Contagem de transações por usuario
user_transactions_count = transaction_dataset.groupby('UserID').size().reset_index(name='transaction_count')
user_transactions_count.sort_values(by='transaction_count', ascending=False).head(5)

,UserID,transaction_count
88310,15885000,7198
242297,9633000,2964
150080,3154000,2897
109940,17188299000,2430
238672,9500204000,1919


In [23]:
browser_transactions_count = identity_dataset.groupby('Browser').size().reset_index(name='transaction_count')
browser_transactions_count.sort_values(by='transaction_count', ascending=False).head()

,Browser,transaction_count
0,Chrome,76059
6,Safari,37281
3,Internet Explorer,10018
2,Firefox,7017
1,Edge,6401


In [24]:
# Merge dos datasets
full_dataset = pd.merge(transaction_dataset, identity_dataset, on='TransactionID', how='inner')

In [25]:
# Tamanho dos datasets
print('Transaction Dataset:', transaction_dataset.shape)
print('Identity Dataset:', identity_dataset.shape)
print('Full Dataset:', full_dataset.shape)

Transaction Dataset: (590540, 11)
Identity Dataset: (144233, 5)
Full Dataset: (144233, 15)


In [26]:
print(f"Quantas transações fraudulentas existem no dataset mergeado?\n{full_dataset.isFraud.value_counts().sort_values(ascending=False)}")

Quantas transações fraudulentas existem no dataset mergeado?
isFraud
0    132915
1     11318
Name: count, dtype: int64
